In [7]:
import numpy as np 
import os
from xml.dom.minidom import parse

In [8]:
cls_to_idx={'RBC':0,'WBC':1,'Platelets':2}

In [9]:
with open('BCCD_Dataset-master/BCCD/ImageSets/Main/train.txt','r') as file:
    TrainLines1=file.readlines()

with open('BCCD_Dataset-master/BCCD/ImageSets/Main/trainval.txt','r') as file:
    TrainLines2=file.readlines()

TrainLines=TrainLines1+TrainLines2
TrainLines=[line.strip() for line in TrainLines]

In [10]:
def cord_converter(size,img_box):
    x1=int(img_box[0])
    y1=int(img_box[1])
    x2=int(img_box[2])
    y2=int(img_box[3])

    dw=np.float32(1/int(size[0]))
    dh=np.float32(1/int(size[1]))
    
    w=x2-x1
    h=y2-y1 
    x=x1+(w/2)
    y=y1+(h/2)

    x=x*dw
    y=y*dh
    w=w*dw
    h=h*dw 

    return [x,y,w,h]

In [11]:
def save_file(picName,size, img_box):
    path='BCCD_Dataset-master/BCCD/labels/train/'+picName+'.txt'
    file=open(path,'a+')

    for box in img_box:
        cls_num=cls_to_idx[box[0]]
        new_box=cord_converter(size, box[1:])
        file.write(f"{cls_num} {new_box[0]} {new_box[1]} {new_box[2]} {new_box[3]}\n")

    file.flush()
    file.close()

In [12]:
def get_xml_data(file):
    dom=parse(file)
    root=dom.documentElement

    img_name=root.getElementsByTagName('filename')[0].childNodes[0].data
    img_size=root.getElementsByTagName('size')[0]

    img_w=img_size.getElementsByTagName('width')[0].childNodes[0].data
    img_h=img_size.getElementsByTagName('height')[0].childNodes[0].data
    img_c=img_size.getElementsByTagName('depth')[0].childNodes[0].data

    objects=root.getElementsByTagName('object')
    img_box=[]

    for box in objects:
        cls_name=box.getElementsByTagName('name')[0].childNodes[0].data
        x1=box.getElementsByTagName('xmin')[0].childNodes[0].data
        y1=box.getElementsByTagName('ymin')[0].childNodes[0].data
        x2=box.getElementsByTagName('xmax')[0].childNodes[0].data
        y2=box.getElementsByTagName('ymax')[0].childNodes[0].data
        img_box.append([cls_name,x1,y1,x2,y2])

    save_file(img_name.strip('.jpg'),[img_w,img_h],img_box)

In [13]:
files=os.listdir('BCCD_Dataset-master/BCCD/Annotations/')

for file in files:
    if file.strip('.xml') in TrainLines:
        file=os.path.join('BCCD_Dataset-master/BCCD/Annotations/',file)
        get_xml_data(file)

In [14]:
from ultralytics import YOLO
import torch

In [15]:
# model=YOLO('yolov8n.yaml')
# model.to(torch.device('cuda'))
# results=model.train(data='BCCD_Dataset-master\BCCD\config.yaml', epochs=100)

In [16]:
!yolo task=detect mode=predict model=runs\detect\train2\weights\best.pt conf=0.25 source=BCCD_Dataset-master\BCCD\images\test

Ultralytics YOLOv8.0.172  Python-3.9.17 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050, 8191MiB)
YOLOv8n summary (fused): 168 layers, 3006233 parameters, 0 gradients

image 1/159 d:\Coding\ML\MLCourseCodes\MLProjects\YOLOV8BCCD\BCCD_Dataset-master\BCCD\images\test\BloodImage_00000.jpg: 480x640 21 RBCs, 1 WBC, 132.2ms
image 2/159 d:\Coding\ML\MLCourseCodes\MLProjects\YOLOV8BCCD\BCCD_Dataset-master\BCCD\images\test\BloodImage_00002.jpg: 480x640 23 RBCs, 1 WBC, 8.0ms
image 3/159 d:\Coding\ML\MLCourseCodes\MLProjects\YOLOV8BCCD\BCCD_Dataset-master\BCCD\images\test\BloodImage_00007.jpg: 480x640 14 RBCs, 2 WBCs, 1 Platelets, 9.0ms
image 4/159 d:\Coding\ML\MLCourseCodes\MLProjects\YOLOV8BCCD\BCCD_Dataset-master\BCCD\images\test\BloodImage_00011.jpg: 480x640 20 RBCs, 2 WBCs, 9.0ms
image 5/159 d:\Coding\ML\MLCourseCodes\MLProjects\YOLOV8BCCD\BCCD_Dataset-master\BCCD\images\test\BloodImage_00014.jpg: 480x640 19 RBCs, 4 WBCs, 2 Plateletss, 8.0ms
image 6/159 d:\Coding\ML\MLCourseCodes\MLProject